In [41]:
!pip install py3-tts SpeechRecognition langchain langchain-community pyaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.6/738.6 kB 264.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 662.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.5/388.5 kB 370.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 527.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 508.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 749.0 kB/s eta 0:00:00--:--
  Attempting uninstall: pyobjc-core
    Found existing installation: pyobjc-core 10.2
    Uninstalling pyobjc-core-10.2:
      Successfully uninstalled pyobjc-core-10.2
  Attempting uninstall: pyobjc-framework-libdispatch
    Found existing installation: pyobjc-framework-libdispatch 10.2
    Uninstalling pyobjc-framework-libdispatch-10.2:
      Successfully uninstalled pyobjc-framework-libdispatch-10.2
  Attempting uninstall: pyobjc-framework-Cocoa
    Found exis

In [1]:
import speech_recognition as sr
import pyttsx3

from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ChatMessageHistory

In [2]:
# STT
print("Say something!")
def human():
    recognizer = sr.Recognizer()
    
    with sr.Microphone() as source:
        audio = recognizer.listen(source)
        
        try:
            text = recognizer.recognize_google(audio)
            print(f"You said: {text}")
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio")
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service: {e}")
            
    return text

# human_output = human()
# human_output

Say something!


In [3]:
# AI Prompt Template
llm = Ollama(model="llama2")
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a chatbot assistant. Keep your responses concise and relevant. Do not start responses with either 'Assisstant:' or 'Bot:' or 'AI:'."),
    MessagesPlaceholder(variable_name="chat_history")
])
output_parser = StrOutputParser()
chat_history_memory = ChatMessageHistory()

# Chain
chain = prompt | llm | output_parser

In [4]:
def update_chat_history(text, human=True):
    if human:
        chat_history_memory.add_user_message(text)
    else:
        chat_history_memory.add_ai_message(text)

In [5]:
# TTS
def ai(chain, chat_history_memory):
    response = chain.invoke({"chat_history": chat_history_memory.messages})
    if response[:3] == "AI:": # AI seems to fail to respect the prompt template. Not sure why.
        response = response[3:]
    print(f"Assistant said: {response}")
    
    engine = pyttsx3.init()
    engine.say(response)
    engine.runAndWait()
    engine.stop()
    
    return response

# ai_output = ai(chain, human_output)
# ai_output

In [ ]:
# Flow
human_output, ai_output = "", ""

print("Transcript")
while True:
    human_output = human()
    if human_output == "thanks bye":
        break
    update_chat_history(human_output)
    ai_output = ai(chain, chat_history_memory)
    update_chat_history(ai_output, human=False)

In [ ]:
chat_history_memory

In [33]:
chat_history_memory.clear()

### TODO
1. Create a history aware chain. Append both AI and Human responses into the chat history.
2. Add AI response TTS.
3. Create a flow of events for STT to TTS.
4. Add Retriever (Future) https://python.langchain.com/v0.1/docs/use_cases/chatbots/quickstart/
5. Save chat history. keep updating and make it feel like its memory. Load whenever a user logs on.